In [20]:
import re

# query scriptttt
query = """

--WITH dates as (													
SELECT DISTINCT date_id as grass_date													
FROM regbida_keyreports.dim_date													
WHERE (date_id BETWEEN DATE('2023-01-01') and DATE('2023-06-30'))													
)													
, sab as (													
SELECT DISTINCT promotionid													
FROM phbi_com.bm_vouchers													
WHERE is_seller_absorbed = TRUE													
)													
, pv AS (													
SELECT DISTINCT orderid													
, pv_promotionid as pvid													
, DATE_TRUNC('month', g2n.grass_date) as month_date													
FROM phbi_com.cost_rev_g2n as g2n													
INNER JOIN dates													
on g2n.grass_date = dates.grass_date													
WHERE is_shopid_excluded = 0 and pv_promotionid IS NOT NULL													
)													
, fsv AS (													
SELECT DISTINCT orderid													
, fsv_promotionid as fsvid													
, DATE_TRUNC('month', g2n.grass_date) as month_date													
FROM phbi_com.cost_rev_g2n as g2n													
INNER JOIN dates													
on g2n.grass_date = dates.grass_date													
WHERE is_shopid_excluded = 0 and fsv_promotionid IS NOT NULL													
)													
, sv AS (													
SELECT DISTINCT orderid													
, sv_promotionid as svid													
, DATE_TRUNC('month', g2n.grass_date) as month_date													
FROM phbi_com.cost_rev_g2n as g2n													
INNER JOIN dates													
on g2n.grass_date = dates.grass_date													
WHERE is_shopid_excluded = 0 and sv_promotionid IS NOT NULL													
)													
													
-- Order Revenue													
, local_sip_revenue as (													
SELECT DISTINCT order_id as orderid													
, commission_fee_pct as comm_fee_perc													
, service_fee_pct as serv_fee_perc													
, cc_fee_pct as cc_fee_perc													
FROM mp_cb.ads_sip_local_revenue_1d__ph_live as sip													
INNER JOIN dates													
on DATE(sip.order_create_datetime) = dates.grass_date													
)													
, sip_orders as (													
SELECT DISTINCT order_id as orderid													
, lsp.comm_fee_perc													
, lsp.serv_fee_perc													
, lsp.cc_fee_perc													
FROM mp_order.dwd_order_item_all_ent_df__ph_s0_live as mp													
INNER JOIN dates													
on dates.grass_date = date(from_unixtime(mp.create_timestamp))													
INNER JOIN local_sip_revenue as lsp													
on mp.order_id = lsp.orderid													
WHERE (bi_exclude_reason is null) and is_cb_shop=1													
)													
, sip_rev as (													
SELECT g2n.grass_date													
, g2n.is_net													
, g2n.orderid													
, g2n.userid													
, g2n.shopid													
-- SIP Portion: Already applied here are net logic for SIP orders													
, SUM(CASE WHEN g2n.is_net = 1 THEN COALESCE((sip.cc_fee_perc * g2n.gmv_usd), 0)/(1+coalesce(vat.cb_handling_fee_revenue_vat,0.12)) ELSE 0 END) as sip_transaction_fee													
, SUM(CASE WHEN g2n.is_net = 1 THEN COALESCE((sip.comm_fee_perc * g2n.comm_base_amount_usd), 0)/(1+coalesce(vat.cb_comm_revenue_vat,0.12)) ELSE 0 END) as sip_commission_fee													
, SUM(CASE WHEN g2n.is_net = 1 THEN COALESCE((sip.serv_fee_perc * g2n.comm_base_amount_usd), 0)/(1+coalesce(vat.cb_comm_revenue_vat,0.12)) ELSE 0 END) as sip_service_fee													
													
-- NON SIP Portion													
, 0 as non_sip_transaction_fee													
, 0 as non_sip_commission_fee													
, 0 as non_sip_fss_fee													
, 0 as non_sip_ccb_fee													
, 0 as non_sip_others_fee													
FROM phbi_com.cost_rev_g2n as g2n													
INNER JOIN sip_orders as sip													
on g2n.orderid = sip.orderid													
LEFT JOIN phbi_com.vat_exclusion_rates as vat													
on g2n.grass_date = vat.grass_date													
GROUP BY 1,2,3,4,5													
)													
, non_sip_rev as (													
SELECT g2n.grass_date													
, g2n.is_net													
, g2n.orderid													
, g2n.userid													
, g2n.shopid													
-- SIP Portion: Already applied here are net logic for SIP orders													
, 0 as sip_transaction_fee													
, 0 as sip_commission_fee													
, 0 as sip_service_fee													
													
-- NON SIP Portion													
, SUM(COALESCE(transaction_fee_usd_g2n, 0)) as non_sip_transaction_fee													
, SUM(COALESCE(commission_fee_usd_g2n, 0)) as non_sip_commission_fee													
, SUM(COALESCE(fss_fee_usd_g2n, 0)) as non_sip_fss_fee													
, SUM(COALESCE(ccb_fee_usd_g2n, 0)) as non_sip_ccb_fee													
, SUM(COALESCE(service_fee_usd_g2n, 0)													
- COALESCE(fss_fee_usd_g2n, 0)													
- COALESCE(ccb_fee_usd_g2n, 0)) as non_sip_others_fee													
FROM phbi_com.cost_rev_g2n as g2n													
INNER JOIN dates													
on g2n.grass_date = dates.grass_date													
LEFT JOIN sip_rev as sip													
on g2n.orderid = sip.orderid													
WHERE sip.orderid is NULL and is_shopid_excluded = 0													
GROUP BY 1,2,3,4,5													
)													
, ord_rev_temp as (													
SELECT * FROM sip_rev													
UNION ALL													
SELECT * FROM non_sip_rev													
)													
, ord_rev as (													
SELECT orderid													
, SUM(non_sip_commission_fee) as non_sip_commission_fee													
, SUM(non_sip_fss_fee) as non_sip_fss_fee													
, SUM(non_sip_ccb_fee) as non_sip_ccb_fee													
, SUM(non_sip_others_fee) as non_sip_others_fee													
, SUM(non_sip_transaction_fee) as non_sip_transaction_fee													
, SUM(sip_commission_fee) as sip_commission_fee													
, SUM(sip_service_fee) as sip_service_fee													
, SUM(sip_transaction_fee) as sip_transaction_fee													
, SUM(non_sip_transaction_fee + sip_transaction_fee) as transaction_fee													
, SUM(non_sip_commission_fee + sip_commission_fee) as commission_fee													
, SUM(non_sip_fss_fee + non_sip_ccb_fee + non_sip_others_fee + sip_service_fee) as service_fee													
FROM ord_rev_temp													
GROUP BY 1													
)													
-- Partner Paid and Co-Sponsored Vouchers													
, csv as (													
SELECT DISTINCT TRY(DATE(DATE_PARSE(month_date, '%m/%d/%Y'))) as month_date													
, CAST(promoid as BIGINT) as promoid													
, CAST(1.0 AS DECIMAL) - CAST(non_shp_percentage as DECIMAL(3,1)) as shp_percentage													
, CAST(non_shp_percentage as DECIMAL(3,1)) as csv_percentage													
FROM phbi_com.weekly_pc1pc2_csvpp_vouchers													
WHERE ingestion_timestamp = (SELECT max(ingestion_timestamp) as mx FROM phbi_com.weekly_pc1pc2_csvpp_vouchers)													
)													
-- Order Cost													
, ord_cost AS (													
SELECT DATE_TRUNC('month', g2n.grass_date) as month_date													
, g2n.grass_date													
, is_net													
, g2n.orderid													
, userid													
, shopid													
, pv.pvid													
, fsv.fsvid													
, sv.svid													
, CASE WHEN (csv_fsv.promoid IS NOT NULL OR csv_pv.promoid IS NOT NULL OR csv_sv.promoid IS NOT NULL) THEN 1 ELSE 0 END AS is_cosponsored_order													
, SUM(gmv_usd) as gmv_usd													
, SUM(order_fraction) as orders													
, SUM(comm_base_amount_usd) as comm_base													
													
-- COSTS:													
, SUM(CASE WHEN csv_fsv.promoid IS NOT NULL THEN (COALESCE(logistics_cost_g2n, 0)*csv_fsv.shp_percentage)													
ELSE COALESCE(logistics_cost_g2n, 0) END) as log_cost													
, SUM(CASE WHEN csv_pv.promoid IS NOT NULL THEN (COALESCE(pv_voucher_cost_g2n, 0)*csv_pv.shp_percentage)													
ELSE COALESCE(pv_voucher_cost_g2n, 0) END) as pv_voucher_cost													
, SUM(CASE WHEN csv_sv.promoid IS NOT NULL THEN (COALESCE(sv_voucher_cost_g2n, 0)*csv_sv.shp_percentage)													
ELSE COALESCE(sv_voucher_cost_g2n, 0) END) as sv_voucher_cost													
, SUM(COALESCE(voucher_cost_g2n, 0)) as voucher_cost													
, SUM(COALESCE(coins_cost_g2n,0)) as coins_cost													
, SUM(COALESCE(item_cost_g2n, 0)) as item_cost													
FROM phbi_com.cost_rev_g2n as g2n													
INNER JOIN dates													
on g2n.grass_date = dates.grass_date													
LEFT JOIN pv													
on g2n.orderid = pv.orderid													
LEFT JOIN fsv													
on g2n.orderid = fsv.orderid													
LEFT JOIN sv													
on g2n.orderid = sv.orderid													
LEFT JOIN csv as csv_fsv													
on fsv.fsvid = csv_fsv.promoid													
and fsv.month_date = csv_fsv.month_date													
LEFT JOIN csv as csv_pv													
on pv.pvid = csv_pv.promoid													
and pv.month_date = csv_pv.month_date													
LEFT JOIN csv as csv_sv													
on sv.svid = csv_sv.promoid													
and sv.month_date = csv_sv.month_date													
WHERE is_shopid_excluded = 0													
GROUP BY 1,2,3,4,5,6,7,8,9,10													
)													
SELECT oc.*													
, ore.non_sip_commission_fee													
, ore.non_sip_fss_fee													
, ore.non_sip_ccb_fee													
, ore.non_sip_others_fee													
, ore.non_sip_transaction_fee													
, ore.sip_commission_fee													
, ore.sip_service_fee													
, ore.sip_transaction_fee													
, ore.transaction_fee													
, ore.commission_fee													
, ore.service_fee													
FROM ord_cost as oc													
LEFT JOIN ord_rev as ore													
on oc.orderid = ore.orderid													
"""



In [21]:
#CTE scrapper non datamart schema
# regex
# cte_name_pattern1 = re.split(r',\s*(?=\w+\s+AS\s*\()', flags=re.IGNORECASE)  #not correct
# cte_name_pattern2 = r',\s*(?=\w+\s+AS\s*\"  # not correct
cte_name_pattern = r"\b(\w+)\s+AS\s*\("
comment_pattern = r"--.*?$"  # Pattern to match comments starting with (--)

# Remove comments from the query
query_without_comments = re.sub(comment_pattern, "", query, flags=re.MULTILINE)

# Find all CTE names using the pattern
cte_names = re.findall(cte_name_pattern, query_without_comments, re.IGNORECASE)

# Print the extracted CTE names
print("Extracted CTE names:", cte_names)

Extracted CTE names: ['sab', 'pv', 'fsv', 'sv', 'local_sip_revenue', 'sip_orders', 'sip_rev', 'non_sip_rev', 'ord_rev_temp', 'ord_rev', 'csv', 'ord_cost']


In [22]:
#version #0

# count how many times it was called from FROM clause and JOIN clause

for i in cte_names:
    print(i, query.count(i) - 1) # -1 because the first one is the CTE itself

sab 0
pv 19
fsv 16
sv 61
local_sip_revenue 1
sip_orders 1
sip_rev 6
non_sip_rev 1
ord_rev_temp 1
ord_rev 3
csv 24
ord_cost 1


In [23]:
#version #1 - not correct
for cte_name in cte_names:
    # pattern = r'\b{}\b(?!\.)'.format(re.escape(cte_name))
    pattern = r'(?<!\.)\b{}\b(?!\.)'.format(re.escape(cte_name))
    count = len(re.findall(pattern, query))
    print(cte_name, count - 1 if count > 0 else 0)



sab 0
pv 1
fsv 1
sv 1
local_sip_revenue 1
sip_orders 1
sip_rev 2
non_sip_rev 1
ord_rev_temp 1
ord_rev 1
csv 3
ord_cost 1


In [24]:
# version #2 ignores commented lines but not AS and math functions

# Function to remove commented lines
def remove_comments(query):
    return '\n'.join([line for line in query.splitlines() if not line.strip().startswith('--')])

# Remove comments from the query
clean_query = remove_comments(query)

# Count occurrences of each CTE name
for cte_name in cte_names:
    pattern = r'(?<!\.)\b{}\b(?!\.)'.format(re.escape(cte_name))
    count = len(re.findall(pattern, clean_query))
    print(cte_name, count - 1 if count > 0 else 0)


sab 0
pv 1
fsv 1
sv 1
local_sip_revenue 1
sip_orders 1
sip_rev 2
non_sip_rev 1
ord_rev_temp 1
ord_rev 1
csv 3
ord_cost 1


In [25]:
#completed scrapper of counting cte version #3

# Function to remove commented lines
def remove_comments(query):
    return '\n'.join([line for line in query.splitlines() if not line.strip().startswith('--')])

# Remove comments from the query
clean_query = remove_comments(query)

# Regex pattern to exclude specific contexts
def create_pattern(cte_name):
    return r'(?<!\.)\b{}\b(?!\.)'.format(re.escape(cte_name))

# Function to filter out invalid occurrences
def is_valid_occurrence(cte_name, match, query):
    invalid_patterns = [
        r'\bAS\s+{}'.format(re.escape(cte_name)),         # After AS
        r'{}\s*/'.format(re.escape(cte_name)),            # Division operation
        r'{}\s*\+'.format(re.escape(cte_name)),           # Addition operation
        r'{}\s*-'.format(re.escape(cte_name)),            # Subtraction operation
        r'{}\s*\*'.format(re.escape(cte_name)),           # Multiplication operation
        r'/\s*{}'.format(re.escape(cte_name)),            # Division operation
        r'\+\s*{}'.format(re.escape(cte_name)),           # Addition operation
        r'-\s*{}'.format(re.escape(cte_name)),            # Subtraction operation
        r'\*\s*{}'.format(re.escape(cte_name)),           # Multiplication operation
    ]
    start, end = match.span()
    for pattern in invalid_patterns:
        if re.search(pattern, query[start-10:end+10]):
            return False
    return True

# Count occurrences of each CTE name
for cte_name in cte_names:
    pattern = create_pattern(cte_name)
    matches = re.finditer(pattern, clean_query)
    valid_count = sum(1 for match in matches if is_valid_occurrence(cte_name, match, clean_query))
    print(cte_name, valid_count - 1 if valid_count > 0 else 0)


sab 0
pv 1
fsv 1
sv 1
local_sip_revenue 1
sip_orders 1
sip_rev 2
non_sip_rev 1
ord_rev_temp 1
ord_rev 1
csv 3
ord_cost 1
